In [1]:
#Use 
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
from scipy.stats import linregress
from pprint import pprint
from geopy.geocoders import Nominatim

input_data_file = "output/clean_abandon_data.csv"
output_data_file = "output/newark_abandoned_data.csv"


In [2]:
clean_abandon_df = pd.read_csv(input_data_file, encoding="ISO-8859-1")
clean_abandon_df


,ID,Lot,Block,Open,Owner Name,Address,City,State,Zip Code,Most Recent Inspection
0,1,3.0,118.00,Vacant Only,LIP LIFE REALTY INC C/O BYUNG JUN,1011 BROAD ST,Newark,NJ,7102,2017-09-27T00:00:00
1,2,11.1,1183.01,Vacant Only,"JERONIMO,RICARDO L BIGALIA,TAOANE",27-29 SUMO VILLAGE CT,Newark,NJ,7114,2016-10-19T00:00:00
2,3,33.0,1185.01,Vacant Only,BALAN INVESTMENTS LLC,8-10 GOBLE ST,Newark,NJ,7114,2020-02-20T00:00:00
3,4,41.0,121.00,Abandoned,493 WASHINGTON ST LLC,493 WASHINGTON ST,Newark,NJ,7102,2016-10-27T00:00:00
4,5,54.0,123.00,Vacant Only,VULCAN PIONEERS OF N J INC,97-99 CLINTON AVE,Newark,NJ,7114,2020-01-30T00:00:00
...,...,...,...,...,...,...,...,...,...,...
466,496,11.0,3580.00,Vacant Only,"MCCALL, RUTH % COMM MTGE CO",10-12 GRANT AVE,Newark,NJ,7112,2020-01-08T00:00:00
467,497,16.0,3582.00,Vacant Only,CITY OF NEWARK,145 WATSON AVE,Newark,NJ,7112,2019-11-14T00:00:00
468,498,13.0,3583.00,Vacant Only,"FDF 3 LLC,",916 HUNTERDON ST,Newark,NJ,7112,2018-08-27T00:00:00
469,499,13.0,3583.00,Vacant Only,"FDF 3 LLC,",916 HUNTERDON ST,Newark,NJ,7112,2020-08-27T00:00:00


In [3]:
#Get the latitude and logitude for each address

#define variables
indices = []
lat_list = []
lon_list = []
drop_indices = []

#Grab indicies for loopin'
indices = list(clean_abandon_df.index.values)


In [6]:
geolocator = Nominatim(user_agent="geoapiExercises")

location = geolocator.geocode(f"{clean_abandon_df['Address'][0]},{clean_abandon_df['City'][0]},{clean_abandon_df['State'][0]}")
data = location.raw
loc_data = data['display_name'].split()
zip_data = loc_data[-3].replace(",","")
zip_data

'07102'

In [ ]:
# initialize Nominatim API (geopy.geocoders)
geolocator = Nominatim(user_agent="Nominatim")

#Loop it
for index in indices:
    try:
        location = geolocator.geocode(f"{clean_abandon_df['Address'][index]},{clean_abandon_df['City'][index]},{clean_abandon_df['State'][index]}")
        lat_list.append(location.latitude)
        lon_list.append(location.longitude)
    except:
        lat_list.append("NULL")
        lon_list.append("NULL")
        drop_indices.append(index)

In [ ]:
#Put lat/lon data in dictionary
lon_lat_dict = {'Latitude':lat_list, 'Longitude':lon_list}

#Put lat/lon data in DataFrame
lon_lat_df = pd.DataFrame(lon_lat_dict)

#Join lat/lon data and clean_abandon_df in a new DataFrame 
lon_lat2_df = clean_abandon_df.join(lon_lat_df, how='left')

#Drop 'NULL' results from new DataFrame
newark_abandoned_df = lon_lat2_df.drop(drop_indices, inplace=False)


In [4]:
#"On screen"
newark_abandoned_df

#Export to CSV
newark_abandoned_df.to_csv(output_data_file, index=False, header=True)